# CDR Experiments via Pymagicc
##### CC-BY 4.0 2020 @safiume

In [ ]:
# NBVAL_IGNORE_OUTPUT
import sys, re
from os import listdir
from os.path import join, dirname
from pprint import pprint
import expectexception
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline
import f90nml

In [2]:
#Disable warning supression if running locally
import warnings
warnings.simplefilter("ignore", category=Warning, lineno=0, append=False)

In [3]:
import pymagicc
from pymagicc import MAGICC6
from pymagicc.io import MAGICCData, read_cfg_file, NoReaderWriterError
from pymagicc.scenarios import ( rcp26, rcp45, rcps )
print(pymagicc.__version__)
print(sys.version)
rcp26.__class__.__bases__

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


2.0.0b14+19.g3c7f351.dirty
3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) 
[Clang 6.0 (clang-600.0.57)]


(scmdata.dataframe.ScmDataFrame,)

In [4]:
## list declines to run
declines = ["emax", "lmax", "emin", "lmin"]
SCEN_DIR = "SCEN"
MAGICC6_DIR = join("..", "CDRRUN")

##Start of Magic settings
magcfg = { "startyear" : 1765,
          "endyear" : 2500,
          "core_climatesensitivity" : 3,
          "co2_tempfeedback_yrstart" : 2005,
          "co2_fertilization_yrstart" : 2005,
          "co2_switchfromconc2emis_year" : 2005,
          "rf_mhalo_constantafteryr" : 2500,
          "rf_fgas_constantafteryr" : 2500,
          "out_forcing" : 1,
          "out_emissions" : 1,
          "out_concentrations" : 1 }
### End MAGICC Settings

Import IMAGE-SSP1-1.9 from manually imported SCEN built from 
"IMAGE 3.0.1","SSP1-19","World","Diagnostics|MAGICC6|Harmonized Input|*

© IAMC 1.5°C Scenario Explorer hosted by IIASA https://data.ene.iiasa.ac.at/iamc-1.5c-explorer

There are some artifacts in the dataset and subsequent graphs as the download was missing the full fgasses and halo gasses.

In [5]:

for name in (declines):
    declinefile = join(SCEN_DIR, "ONC" + name + ".SCEN")
    oncdata = MAGICCData(declinefile,
                             columns={"model": ["CDRex"], "scenario": [name]}, inplace=True)
    rcps = rcps.append(oncdata)

img119 = MAGICCData(join(SCEN_DIR, 'imgs119.SCEN'), 
                    columns={"model": ["IMAGE"], "scenario": ['SSP119']}, inplace=True)
rcps = rcps.append(img119)
print(rcps["scenario"].unique())
#image119 = image119.assign(region='WORLD').set_index('climate_model', append=True)
rcps.filter(scenario='lmin').timeseries().head(3)

['emax' 'emin' 'lmax' 'lmin' 'RCP26' 'SSP119']


time                                                                                            2005-01-01  \
model scenario region variable                                   unit       climate_model todo               
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr  unspecified   SET        1.180   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr  unspecified   SET        8.030   
                      Emissions|BC                               Mt BC / yr unspecified   SET        8.251   

time                                                                                            2010-01-01  \
model scenario region variable                                   unit       climate_model todo               
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr  unspecified   SET       1.3732   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr  unspecified   SET       9.1209   
                      Emissions|BC                               Mt BC / yr unspecified   SET       8.6231   

time                                                                                            2011-01-01  \
model scenario region variable                                   unit       climate_model todo               
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr  unspecified   SET       1.3204   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr  unspecified   SET       9.4954   
                      Emissions|BC                               Mt BC / yr unspecified   SET       8.6231   

time                                                                                            2012-01-01  \
model scenario region variable                                   unit       climate_model todo               
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr  unspecified   SET       1.3786   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr  unspecified   SET       9.6670   
                      Emissions|BC                               Mt BC / yr unspecified   SET       8.6231   

time                                                                                            2013-01-01  \
model scenario region variable                                   unit       climate_model todo               
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr  unspecified   SET       1.4088   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr  unspecified   SET       9.7651   
                      Emissions|BC                               Mt BC / yr unspecified   SET       8.6231   

time                                                                                            2014-01-01  \
model scenario region variable                                   unit       climate_model todo               
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr  unspecified   SET       1.3788   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr  unspecified   SET       9.8475   
                      Emissions|BC                               Mt BC / yr unspecified   SET       8.6231   

time                                                                                            2015-01-01  \
model scenario region variable                                   unit       climate_model todo               
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr  unspecified   SET       1.5191   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr  unspecified   SET       9.8306   
                      Emissions|BC                               Mt BC / yr unspecified   SET       8.6231   

time                                                                                            2016-01-01  \
model scenario region variable   

In [7]:
#GCB cumulative in GtC to 1913 when CO₂ was 300.2 ppm per 
#Etheridge, D.M. et al. (2010), Law Dome Ice Core 
#2000-Year CO2, CH4, and N2O Data, IGBP PAGES/World Data 
#Center for Paleoclimatology, Data Contribution Series 
# 2010-070, NOAA/NCDC Paleoclimatology Program, Boulder CO, USA
#http://cdiac.ess-dive.lbl.gov/trends/co2/ice_core_co2.html 
#ftp.ncdc.noaa.gov/pub/data/paleo/icecore/antarctica/law/law2006.txt

#https://doi.org/10.1002/essoar.10503117.1, see data
#https://doi.org/10.5194/essd-10-405-2018, see data & sum till 1913
FF = 21.8
LuC = 57.4
lndsnk = 20.6
CCCanthroC = FF + LuC - lndsnk
PeakAnthroC = 500 

PeakCyr = 2021
Bkyr = 2050
RemainingYrs = 2100 - Bkyr
Peak_to_Bk = Bkyr - PeakCyr




In [9]:
CCCx2050 = rcps.filter(scenario='lmin')
src_df = CCCx2050.timeseries()
#kigali_yrs = 2019,2024,2025,2029,2034,2036
#ki_mo_gasses_F = HFC134a, HFC245fa, HFC277ea, HFC4310, HFC32, HFC125, HFC143a, 
#HFC23
#sf6-eu 30% by 2030
source_cols = CCCx2050.timeseries().columns
idxs = list(map(lambda x: 11+x*10, range(9)))
idxs.append(len(source_cols)-1)
idxs.insert(0,0)
idxs.insert(1,1)

idxs.insert(2,10)
idxs.insert(3,15)
idxs.insert(4,16)
idxs.insert(5,20)
idxs.insert(7,25)
idxs.insert(8,27)


newcols = []
for n in idxs:
    newcols.append(source_cols[n])
CCC_df = src_df[newcols]



#rewrite CCCx2050 from timeseries in line2 w/ Slices of cols in newcols
# and wrap in MAGICCData
CCCx2050 = MAGICCData(CCC_df)
CCCx2050.timeseries()

time                                                                                                  2005-01-01 00:00:00  \
model scenario region variable                                   unit             climate_model todo                        
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr        unspecified   SET                 1.180   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr        unspecified   SET                 8.030   
                      Emissions|BC                               Mt BC / yr       unspecified   SET                 8.251   
                      Emissions|CH4                              Mt CH4 / yr      unspecified   SET               315.903   
                      Emissions|CO                               Mt CO / yr       unspecified   SET              1061.526   
                      Emissions|NH3                              Mt N / yr        unspecified   SET                41.884   
                      Emissions|NOx                              Mt N / yr        unspecified   SET                38.795   
                      Emissions|N2O                              Mt N2ON / yr     unspecified   SET                 7.684   
                      Emissions|NMVOC                            Mt NMVOC / yr    unspecified   SET               215.463   
                      Emissions|OC                               Mt OC / yr       unspecified   SET                36.392   
                      Emissions|SOx                              Mt S / yr        unspecified   SET                56.720   
                      Emissions|C2F6                             kt C2F6 / yr     unspecified   SET                 2.650   
                      Emissions|C6F14                            kt C6F14 / yr    unspecified   SET                 0.436   
                      Emissions|CF4                              kt CF4 / yr      unspecified   SET                11.606   
                      Emissions|HFC125                           kt HFC125 / yr   unspecified   SET                13.759   
                      Emissions|HFC134a                          kt HFC134a / yr  unspecified   SET               120.933   
                      Emissions|HFC143a                          kt HFC143a / yr  unspecified   SET                12.448   
                      Emissions|HFC227ea                         kt HFC227ea / yr unspecified   SET                 4.890   
                      Emissions|HFC23                            kt HFC23 / yr    unspecified   SET                10.812   
                      Emissions|HFC245fa                         kt HFC245fa / yr unspecified   SET                26.889   
                      Emissions|HFC32                            kt HFC32 / yr    unspecified   SET                10.993   
                      Emissions|HFC4310                          kt HFC4310 / yr  unspecified   SET                 3.235   
                      Emissions|SF6                              kt SF6 / yr      unspecified   SET                 6.431   

time                                                                                                  2010-01-01 00:00:00  \
model scenario region variable                                   unit             climate_model todo                        
CDRex lmin     World  Emissions|CO2|MAGICC AFOLU                 Gt C / yr        unspecified   SET                1.3732   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr        unspecified   SET                9.1209   
                      Emissions|BC                               Mt BC / yr       unspecified   SET                8.6231   
                      Emissions|CH4                              Mt CH4 / yr      unspecified   SET              332.0000   
                      Emissions|CO                               Mt CO / yr       unspecified   SET             1068.0009   

# Scenarios

The five RCP scenarios and experimental declines are loaded as DataFrames in pyhector. They contain the following emissions:

In [ ]:
print(rcps["scenario"].unique())
rcps.filter(scenario="SSP119").timeseries().head(5)

In [ ]:
rcp26.head(2)

They are split up in regions:

In [ ]:
rcp26["region"].unique()

They have the following units:

In [ ]:
#pymagicc.units
rcps[["variable", "unit"]].drop_duplicates()
rcp26[["variable"]].head()

In [ ]:
categories_to_plot = rcp26["variable"].drop_duplicates().tolist()

for g in rcp26.filter(
    variable=categories_to_plot,
    year=range(1000, 2150)
).groupby("variable"):
    plt.figure(figsize=(12, 7))
    g.lineplot(hue="region").set_title(g.get_unique_meta("variable", True))

In [ ]:
categories_to_plot = rcp26["variable"].drop_duplicates().tolist()

for g in rcps.filter(
    region="World",
    variable=categories_to_plot,
    year=range(1000, 2150)
).groupby("variable"):
    plt.figure(figsize=(12, 7))
    g.lineplot(hue="scenario").set_title(g.get_unique_meta("variable", True))

# Fossil fuel emissions for the five RCP scenarios and Experimental Declines.

In [ ]:
x = (datetime(2000, 1, 1), datetime(magcfg["endyear"], 1, 1))

pret="OpenCDR & RCP experimental scenarios for "
gtgvar = "*CO2|MAGICC Fossil and Industrial", "*CO2|MAGICC AFOLU*"
mtgvar = "*Emissions|CH4", "*Emissions|CO"
list = [gtgvar,mtgvar]
title = [pret + "Fossil Fuel emissions (CO₂)", 
         pret + "Methane & Carbon Monoxide (CH4, CO)"]

fig, axes = plt.subplots(2,1, figsize = (16, 10))
for n,var in enumerate(list):
    varun = var[(n-1)]
    test = rcps.filter(region="World", variable=var).lineplot(hue="scenario", style="variable", ax=axes[n])
    ylab = rcps.filter(region="World", variable=varun)["unit"].iloc[[0]].all()
    axes[n].set_ylabel(ylab)
    axes[n].set_title(title[n])
    axes[n].set_xlim(x)

## Running MAGICC

A single `pymagicc` run doesn't take long and returns a Pandas Dict.
(If not on Windows, the very first run might be slower due to setting up Wine.)

In [ ]:
%time results = pymagicc.run(rcp26)
results = ""
%time results = pymagicc.run(rcps.filter(scenario="emin"), **magcfg)
results = ""
%time results = pymagicc.run(rcp26, **magcfg)
slice = results.filter(region="World",variable="Radiative Forcing")["unit"].head(3)

In [ ]:
vars = ["Surface Temperature","Radiative Forcing",
        "Atmospheric Concentrations|CO2","Emissions|CO2|MAGICC Fossil and Industrial",
        "Emissions|CO2|MAGICC AFOLU"]
x = (datetime(2000, 1, 1), datetime(magcfg["endyear"], 1, 1))
fig, axes = plt.subplots((len(vars)), 1, figsize=(24, 32))

for sdf in rcps["scenario"].unique():
    results = results.append(pymagicc.run(MAGICCData(rcps.filter(scenario=sdf)), **magcfg))
for n, var in enumerate(vars):
    results.filter(region="World", variable=var).lineplot(hue="scenario", ax=axes[n])
    ylab = results.filter(region="World", variable=var)["unit"].iloc[[0]].all()
    axes[n].set_ylabel(ylab)
    axes[n].set_title(var)
    axes[n].set_xlim(x)


The default parameters are the ones that were used to produce the RCP GHG concentrations (see also http://live.magicc.org/). Of course it's easy to change them.

In [ ]:
low = pymagicc.run(rcp45, core_climatesensitivity=1.5, endyear=magcfg["endyear"])
default = pymagicc.run(rcp45, core_climatesensitivity=3, endyear=magcfg["endyear"])
high = pymagicc.run(rcp45, core_climatesensitivity=4.5, endyear=magcfg["endyear"])

In [ ]:
filtering = {
    "variable": "Surface Temperature",
    "region": "World",
    "year": range(1850, magcfg["endyear"]),
}

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 9))
default.filter(**filtering).line_plot(x="time", ax=ax)
plt.fill_between(
    low.filter(**filtering)["time"].values,
    low.filter(**filtering).timeseries().values.squeeze(),
    high.filter(**filtering).timeseries().values.squeeze(),
    color="lightgray"
)

plt.title(
    "RCP 4.5 with equilibrium climate sensitivity set to 1.5, 3, and 4.5"
)
plt.ylabel("°C");